### Import packages

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
import glob

### Directory and file names to be analyzed

In [ ]:
direct = '/pscratch/sd/u/uo1999/gw170817_sim/att_convergence_010m/att_1.0e-05_0.65_ye'
finaldir = 'att_1.0e-05_0.65_ye'
file = '/allData.h5'
parfile = '/plt09600_particles'
num_particles_per_energy_bin = 92
num_energy_bins = 13

direct = '/pscratch/sd/u/uo1999/gw170817_sim/att_convergence_010m/att_1.0e-05'
finaldir = 'att_1.0e-05'
file = '/allData.h5'
parfile = '/plt09600_particles'
num_particles_per_energy_bin = 92
num_energy_bins = 13

direct = '/pscratch/sd/u/uo1999/gw170817_1.00Ye/classical_transport/378_ppEb'
finaldir = '378_ppEb'
file = '/allData.h5'
parfile = '/plt00960_particles'
num_particles_per_energy_bin = 378
num_energy_bins = 13
time_s = 0.32/1000 # seconds

# direct = '/pscratch/sd/u/uo1999/gw170817_1.00Ye/classical_transport/1506_ppEb'
# finaldir = '1506_ppEb'
# file = '/allData.h5'
# parfile = '/plt00384_particles'
# num_particles_per_energy_bin = 1506
# num_energy_bins = 13
# time_s = 0.32/1000 # seconds

### Plot settings and functions

In [ ]:
# Font settings
mpl.rcParams['font.size'] = 22
mpl.rcParams['font.family'] = 'serif'
mpl.rc('text', usetex=False)

# Tick settings
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.major.pad'] = 8
mpl.rcParams['xtick.minor.size'] = 4

mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.size'] = 7
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.size'] = 4
mpl.rcParams['ytick.minor.width'] = 2

# Axis linewidth
mpl.rcParams['axes.linewidth'] = 2

# Tick direction and enabling ticks on all sides
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

# Function to apply custom tick locators and other settings to an Axes object
def apply_custom_settings(ax, log_scale_y=False):

    if log_scale_y:
        # Use LogLocator for the y-axis if it's in log scale
        ax.set_yscale('log')
        ax.yaxis.set_major_locator(LogLocator(base=10.0))
        ax.yaxis.set_minor_locator(LogLocator(base=10.0, subs='auto', numticks=100))
    else:
        # Use AutoLocator for regular scales
        ax.yaxis.set_major_locator(AutoLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
    
    # Apply the AutoLocator for the x-axis
    ax.xaxis.set_major_locator(AutoLocator())
    ax.xaxis.set_minor_locator(AutoMinorLocator())

def plot_color_map(x, y, z, min_cb, max_cb, x_label, y_label, title, cbar_label, colormap, filename, doshow=True, dosave=True):

    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot pcolormesh
    c = ax.pcolormesh(x, y, z, shading='auto', cmap=colormap, vmin=min_cb, vmax=max_cb)

    # # Add contour lines
    # contour = ax.contour(x, y, z, colors='black', linewidths=1.5, levels=5)
    # ax.clabel(contour, inline=True, fontsize=15, fmt='%1.1f')
    # # contour = ax.contour(x, y, z, colors='black', linewidths=1.5, levels=[1e5, 1e6, 1e7])
    # # ax.clabel(contour, inline=True, fontsize=15, fmt='%1.1e')

    # Plot settings
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    # ax.set_title(title+'\nmin: {:.2e}\nmax: {:.2e}'.format(np.nanmin(z), np.nanmax(z)))
    ax.set_title(title)

    # Add color bar
    cbar = fig.colorbar(c, ax=ax, label=cbar_label)
    cbar.ax.yaxis.set_minor_locator(AutoMinorLocator())

    apply_custom_settings(ax, False)

    # Ensure equal aspect ratio
    ax.set_aspect('equal', 'box')
    
    # ax.set_ylim(0, 32)

    # Save figure
    if dosave:
        fig.savefig(filename, format='png', bbox_inches='tight')

    # Display figure
    if doshow:
        plt.show()
    # display(fig)
    
    # Close figure
    plt.close(fig)

### Define mesh and black hole and boundary mask

In [ ]:
rho_ye_T_h5py = h5py.File(direct+'/rho_Ye_T.hdf5', 'r')

# number of cells
Nx = np.array(rho_ye_T_h5py['/ncellsx'])
Ny = np.array(rho_ye_T_h5py['/ncellsy'])
Nz = np.array(rho_ye_T_h5py['/ncellsz'])
print(f'Nx = {Nx}, Ny = {Ny}, Nz = {Nz}')

# cell size
dx = ( np.array(rho_ye_T_h5py['/xmax_cm']) - np.array(rho_ye_T_h5py['/xmin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsx']) # cm
dy = ( np.array(rho_ye_T_h5py['/ymax_cm']) - np.array(rho_ye_T_h5py['/ymin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsy']) # cm
dz = ( np.array(rho_ye_T_h5py['/zmax_cm']) - np.array(rho_ye_T_h5py['/zmin_cm']) ) / np.array(rho_ye_T_h5py['/ncellsz']) # cm
cellvolume = dx * dy * dz # cm^3
print(f'dx = {dx} cm, dy = {dy} cm, dz = {dz} cm')

# cell centers
xc = np.linspace(dx / 2, dx * (Nx - 0.5), Nx) # cm
yc = np.linspace(dy / 2, dy * (Ny - 0.5), Ny) # cm
zc = np.linspace(dz / 2, dz * (Nz - 0.5), Nz) # cm

# cell centers mesh
Xc, Yc, Zc = np.meshgrid(xc, yc, zc, indexing='ij') # cm

bh_radius = 5.43e+05 # cm
bh_center_x = 48.0e+5 # cm
bh_center_y = 48.0e+5 # cm
bh_center_z = 16.0e+5 # cm

distance_from_bh = np.sqrt( (Xc - bh_center_x)**2 + (Yc - bh_center_y)**2 + (Zc - bh_center_z)**2 )
mask_bh = (distance_from_bh < bh_radius + 1*np.max([dx,dy,dz]))
mask_x = (Xc < dx) | (Xc > (Nx - 1) * dx)
mask_y = (Yc < dy) | (Yc > (Ny - 1) * dy)
mask_z = (Zc < dz) | (Zc > (Nz - 1) * dz)
mask = mask_bh | mask_x | mask_y | mask_z

rho_ye_T_h5py.close()

### Define arrays that containg the cell for which particles data is saved

In [ ]:
# Get the cell indices
cell_file_names = glob.glob(direct + parfile + '/FFI_cell_*_*_*')
cell_file_names = [file_name.split('/')[-1] for file_name in cell_file_names]
x_cell_ind = np.array([int(file_name.split('_')[2]) for file_name in cell_file_names])
y_cell_ind = np.array([int(file_name.split('_')[3]) for file_name in cell_file_names])
z_cell_ind = np.array([int((file_name.split('_')[4]).split('.')[0]) for file_name in cell_file_names])
cell_indices = np.array(list(zip(x_cell_ind, y_cell_ind, z_cell_ind)))
print('Number of cells:', len(cell_indices))
print(f'shape of cell_indices: {cell_indices.shape}')

# Get the cell indices for ix fix but iy and iz varying all available cells
x_idx_slice = 48
mask_yz_slice = cell_indices[:,0] == x_idx_slice # fixing the x index in this value
cell_indices_yz_slice = cell_indices[mask_yz_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_yz_slice[:,1], cell_indices_yz_slice[:,2], color='b')
ax.set_xlabel('$i_y$')
ax.set_ylabel('$i_z$')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)

# Get the cell indices for iy fix but ix and iz varying all available cells
y_idx_slice = 48
mask_xz_slice = cell_indices[:,1] == y_idx_slice # fixing the y index in this value
cell_indices_xz_slice = cell_indices[mask_xz_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_xz_slice[:,0], cell_indices_xz_slice[:,2], color='b')
ax.set_xlabel('$i_x$')
ax.set_ylabel('$i_z$')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)

# Get the cell indices for iz fix but ix and iy varying all available cells
z_idx_slice = 16
mask_xy_slice = cell_indices[:,2] == z_idx_slice # fixing the z index in this value
cell_indices_xy_slice = cell_indices[mask_xy_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_xy_slice[:,0], cell_indices_xy_slice[:,1], color='b')
ax.set_xlabel('$i_x$')
ax.set_ylabel('$i_y$')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)

# Combine all cell indices from the three slices
# cell_indices_all = np.concatenate((cell_indices_yz_slice, cell_indices_xz_slice, cell_indices_xy_slice), axis=0)
cell_indices_all = np.concatenate((cell_indices_xz_slice, cell_indices_xy_slice), axis=0)
cell_indices_all = np.unique(cell_indices_all, axis=0)

### Reading FFI quantities

In [ ]:
sigma = np.full_like(Xc, np.nan)
GnPos = np.full_like(Xc, np.nan)
GnNeg = np.full_like(Xc, np.nan)

for indexpair in cell_indices_all:
        
        i = indexpair[0]
        j = indexpair[1]
        k = indexpair[2]
        
        FFI_file_name = f'/FFI_cell_{i}_{j}_{k}.h5'
        
        with h5py.File(direct + parfile + FFI_file_name, 'r') as f:
            GnNeg[i, j, k] = np.float64(f['/GnNeg_inverse_s'][()])
            GnPos[i, j, k] = np.float64(f['/GnPos_inverse_s'][()])
            sigma[i, j, k] = np.float64(f['/sigma_inverse_s'][()])

### Plotting sigma and the positive and negative integrals of G

In [ ]:
sigma[mask] = np.nan
GnPos[mask] = np.nan
GnNeg[mask] = np.nan

sigma[sigma == 0] = np.nan
GnPos[GnPos == 0] = np.nan
GnNeg[GnNeg == 0] = np.nan

# Find the maximum and minimum of log10(sigma) for the two slices, ignoring NaNs
max_log_sigma = max(
    np.nanmax(np.log10(sigma[:, y_idx_slice, :])),
    np.nanmax(np.log10(sigma[:, :, z_idx_slice]))
)
min_log_sigma = min(
    np.nanmin(np.log10(sigma[:, y_idx_slice, :])),
    np.nanmin(np.log10(sigma[:, :, z_idx_slice]))
)

print("max_log_sigma=", max_log_sigma)
print("min_log_sigma=", min_log_sigma)
# max_log_sigma = None,
# min_log_sigma = None,
max_log_sigma= 8.139360022247965,
min_log_sigma= 3.9342695460570956,

plot_color_map(
    x=Xc[:,y_idx_slice,:]/1.0e5,  # Convert to km
    y=Zc[:,y_idx_slice,:]/1.0e5,  # Convert to km
    z=np.log10(sigma[:,y_idx_slice,:]),
    # z=sigma[:,y_idx_slice,:],
    min_cb=min_log_sigma,
    max_cb=max_log_sigma,
    x_label=r'$x \, (\mathrm{km})$',
    y_label=r'$z \, (\mathrm{km})$',
    title=fr'$y={Yc[0,y_idx_slice,0]/1.0e5}\,\mathrm{{km}}$, $t = {time_s/1e-3:.2f}\,\mathrm{{ms}}$',
    cbar_label=r'$\log(\,\sigma_{FFI} \, [\mathrm{1/s}]\,) \, , \, \sigma_{FFI} = \sqrt{AB}$',
    colormap='viridis',
    # colormap='Oranges',
    filename='plots/'+finaldir+f'_{time_s/1e-3:.2f}ms_sigmaFFI_xz.png',
    doshow=True,
    dosave=True
)

plot_color_map(
    x=Xc[:,:,z_idx_slice]/1.0e5,  # Convert to km
    y=Yc[:,:,z_idx_slice]/1.0e5,  # Convert to km
    z=np.log10(sigma[:,:,z_idx_slice]),
    min_cb=min_log_sigma,
    max_cb=max_log_sigma,
    x_label=r'$x \, (\mathrm{km})$',
    y_label=r'$y \, (\mathrm{km})$',
    title=fr'$z={Zc[0,0,z_idx_slice]/1.0e5}\,\mathrm{{km}}$, $t = {time_s/1e-3:.2f}\,\mathrm{{ms}}$',
    cbar_label=r'$\log(\,\sigma_{FFI} \, [\mathrm{1/s}]\,) \, , \, \sigma_{FFI} = \sqrt{AB}$',
    colormap='viridis',
    # colormap='Oranges',
    filename='plots/'+finaldir+f'_{time_s/1e-3:.2f}ms_sigmaFFI_xy.png',
    doshow=True,
    dosave=True
)

In [ ]:
# Find the maximum and minimum of log10(GnPos) for the two slices, ignoring NaNs
max_log_GnPos = max(
    np.nanmax(np.log10(GnPos[:, y_idx_slice, :])),
    np.nanmax(np.log10(GnPos[:, :, z_idx_slice]))
)
min_log_GnPos = min(
    np.nanmin(np.log10(GnPos[:, y_idx_slice, :])),
    np.nanmin(np.log10(GnPos[:, :, z_idx_slice]))
)
print("Maximum log10(GnPos):", max_log_GnPos)
print("Minimum log10(GnPos):", min_log_GnPos)



plot_color_map(
    x=Xc[:,y_idx_slice,:]/1.0e5,  # Convert to km
    y=Zc[:,y_idx_slice,:]/1.0e5,  # Convert to km
    z=np.log10(GnPos[:,y_idx_slice,:]),
    # z=GnPos[:,y_idx_slice,:],
    min_cb=min_log_GnPos,
    max_cb=max_log_GnPos,
    x_label=r'$x \, (\mathrm{km})$',
    y_label=r'$z \, (\mathrm{km})$',
    title=fr'$y={Yc[0,y_idx_slice,0]/1.0e5}\,\mathrm{{km}}$, $t = {time_s/1e-3:.2f}\,\mathrm{{ms}}$',
    cbar_label='$\log(\,B \, [\mathrm{1/s}]\,) \,,\, B=\int \,G(\hat{n})_{>0}\,d\Omega$',
    colormap='viridis',
    filename='plots/'+finaldir+f'_{time_s/1e-3:.2f}ms_posGN_xz.png',
    doshow=True,
    dosave=True
)

plot_color_map(
    x=Xc[:,:,z_idx_slice]/1.0e5,  # Convert to km
    y=Yc[:,:,z_idx_slice]/1.0e5,  # Convert to km
    z=np.log10(GnPos[:,:,z_idx_slice]),
    min_cb=min_log_GnPos,
    max_cb=max_log_GnPos,
    x_label=r'$x \, (\mathrm{km})$',
    y_label=r'$y \, (\mathrm{km})$',
    title=fr'$z={Zc[0,0,z_idx_slice]/1.0e5}\,\mathrm{{km}}$, $t = {time_s/1e-3:.2f}\,\mathrm{{ms}}$',
    cbar_label='$\log(\,B \, [\mathrm{1/s}]\,) \,,\, B=\int \,G(\hat{n})_{>0}\,d\Omega$',
    colormap='viridis',
    # colormap='Oranges',
    filename='plots/'+finaldir+f'_{time_s/1e-3:.2f}ms_posGN_xy.png',
    doshow=True,
    dosave=True
)

In [ ]:
# Find the maximum and minimum of log10(GnNeg) for the two slices, ignoring NaNs
max_log_GnNeg = max(
    np.nanmax(np.log10(GnNeg[:, y_idx_slice, :])),
    np.nanmax(np.log10(GnNeg[:, :, z_idx_slice]))
)
min_log_GnNeg = min(
    np.nanmin(np.log10(GnNeg[:, y_idx_slice, :])),
    np.nanmin(np.log10(GnNeg[:, :, z_idx_slice]))
)
print("max_log_GnNeg=", max_log_GnNeg)
print("min_log_GnNeg=", min_log_GnNeg)



plot_color_map(
    x=Xc[:,y_idx_slice,:]/1.0e5,  # Convert to km
    y=Zc[:,y_idx_slice,:]/1.0e5,  # Convert to km
    z=np.log10(GnNeg[:,y_idx_slice,:]),
    # z=GnNeg[:,y_idx_slice,:],
    min_cb=min_log_GnNeg,
    max_cb=max_log_GnNeg,
    x_label=r'$x \, (\mathrm{km})$',
    y_label=r'$z \, (\mathrm{km})$',
    title=fr'$y={Yc[0,y_idx_slice,0]/1.0e5}\,\mathrm{{km}}$, $t = {time_s/1e-3:.2f}\,\mathrm{{ms}}$',
    cbar_label='$\log(\,A \, [\mathrm{1/s}]\,) \,,\, A=\int \,G(\hat{n})_{<0}\,d\Omega$',
    colormap='viridis',
    filename='plots/'+finaldir+f'_{time_s/1e-3:.2f}ms_negGN_xz.png',
    doshow=True,
    dosave=True
)

plot_color_map(
    x=Xc[:,:,z_idx_slice]/1.0e5,  # Convert to km
    y=Yc[:,:,z_idx_slice]/1.0e5,  # Convert to km
    z=np.log10(GnNeg[:,:,z_idx_slice]),
    min_cb=min_log_GnNeg,
    max_cb=max_log_GnNeg,
    x_label=r'$x \, (\mathrm{km})$',
    y_label=r'$y \, (\mathrm{km})$',
    title=fr'$z={Zc[0,0,z_idx_slice]/1.0e5}\,\mathrm{{km}}$, $t = {time_s/1e-3:.2f}\,\mathrm{{ms}}$',
    cbar_label='$\log(\,A \, [\mathrm{1/s}]\,) \,,\, A=\int \,G(\hat{n})_{<0}\,d\Omega$',
    colormap='viridis',
    # colormap='Oranges',
    filename='plots/'+finaldir+f'_{time_s/1e-3:.2f}ms_negGN_xy.png',
    doshow=True,
    dosave=True
)